<a href="https://colab.research.google.com/github/choijonghong/transformer/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# PyTorch CNN for CIFAR-10 Classification (Colab Ready)",
        "",
        "이 노트북은 PyTorch를 사용하여 CIFAR-10 데이터셋에 대한 간단한 CNN 모델을 학습하고 평가하는 코드입니다.",
        "",
        "**실행 전:** `런타임(Runtime) -> 런타임 유형 변경(Change runtime type) -> T4 GPU`로 설정하여 GPU를 사용하도록 권장합니다."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "import torch\n",
        "import torch.nn as nn\n",
        "import torch.optim as optim\n",
        "import torchvision\n",
        "import torchvision.transforms as transforms\n",
        "import matplotlib.pyplot as plt\n",
        "import numpy as np\n",
        "from torch.utils.data import DataLoader\n",
        "\n",
        "# 1. 장치 설정\n",
        "device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')\n",
        "print(f\"Using device: {device}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2. 데이터셋 로드 및 전처리 (CIFAR-10)"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# 데이터 전처리 정의: 텐서 변환 및 정규화\n",
        "transform = transforms.Compose(\n",
        "    [\n",
        "        transforms.ToTensor(),\n",
        "        # CIFAR-10의 표준 정규화 (평균: 0.5, 표준편차: 0.5)\n",
        "        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))\n",
        "    ])\n",
        "\n",
        "BATCH_SIZE = 4\n",
        "\n",
        "# 학습 데이터셋 다운로드 및 로드\n",
        "trainset = torchvision.datasets.CIFAR10(\n",
        "    root='./data', train=True, download=True, transform=transform\n",
        ")\n",
        "trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)\n",
        "\n",
        "# 테스트 데이터셋 다운로드 및 로드\n",
        "testset = torchvision.datasets.CIFAR10(\n",
        "    root='./data', train=False, download=True, transform=transform\n",
        ")\n",
        "testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)\n",
        "\n",
        "classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3. CNN 모델 정의 (SimpleNet)"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "class SimpleNet(nn.Module):\n",
        "    def __init__(self):\n",
        "        super().__init__()\n",
        "        # 첫 번째 합성곱 계층 (3x32x32 -> 6x14x14)\n",
        "        self.conv1 = nn.Conv2d(3, 6, 5)\n",
        "        self.pool = nn.MaxPool2d(2, 2)\n",
        "        # 두 번째 합성곱 계층 (6x14x14 -> 16x5x5)\n",
        "        self.conv2 = nn.Conv2d(6, 16, 5)\n",
        "        # 완전 연결 계층 (16*5*5 = 400 -> 120)\n",
        "        self.fc1 = nn.Linear(16 * 5 * 5, 120)\n",
        "        self.fc2 = nn.Linear(120, 84)\n",
        "        # 최종 출력 계층 (10개 클래스)\n",
        "        self.fc3 = nn.Linear(84, 10)\n",
        "\n",
        "    def forward(self, x):\n",
        "        # Conv -> ReLU -> Pool\n",
        "        x = self.pool(torch.relu(self.conv1(x)))\n",
        "        # Conv -> ReLU -> Pool\n",
        "        x = self.pool(torch.relu(self.conv2(x)))\n",
        "        # Flatten (평탄화)\n",
        "        x = torch.flatten(x, 1)\n",
        "        # Fully Connected layers\n",
        "        x = torch.relu(self.fc1(x))\n",
        "        x = torch.relu(self.fc2(x))\n",
        "        x = self.fc3(x)\n",
        "        return x\n",
        "\n",
        "net = SimpleNet()\n",
        "net.to(device) # 모델을 GPU로 이동"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4. 손실 함수와 최적화 도구 정의"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# 교차 엔트로피 손실 함수\n",
        "criterion = nn.CrossEntropyLoss()\n",
        "# SGD 최적화 도구 (학습률 0.001, 모멘텀 0.9)\n",
        "optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)\n",
        "\n",
        "print(\"손실 함수 및 최적화 도구 설정 완료\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5. 모델 학습 (2 Epochs)\n",
        "학습 시간 단축을 위해 에포크(Epoch)를 2로 설정합니다."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "for epoch in range(2):  # 에포크 2회 반복\n",
        "    running_loss = 0.0\n",
        "    for i, data in enumerate(trainloader, 0):\n",
        "        # 데이터 얻기: 입력과 레이블\n",
        "        inputs, labels = data\n",
        "        # 데이터를 GPU로 이동\n",
        "        inputs, labels = inputs.to(device), labels.to(device)\n",
        "\n",
        "        # 변화도(Gradient) 매개변수를 0으로 초기화\n",
        "        optimizer.zero_grad()\n",
        "\n",
        "        # 순전파 -> 역전파 + 최적화\n",
        "        outputs = net(inputs)\n",
        "        loss = criterion(outputs, labels)\n",
        "        loss.backward()\n",
        "        optimizer.step()\n",
        "\n",
        "        # 통계 출력\n",
        "        running_loss += loss.item()\n",
        "        if i % 2000 == 1999:    # 2000 미니배치마다 출력\n",
        "            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')\n",
        "            running_loss = 0.0\n",
        "\n",
        "print('Finished Training')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6. 모델 테스트 및 정확도 평가"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "correct = 0\n",
        "total = 0\n",
        "# 테스트 시에는 변화도(gradient)를 계산할 필요가 없습니다.\n",
        "with torch.no_grad():\n",
        "    for data in testloader:\n",
        "        images, labels = data\n",
        "        images, labels = images.to(device), labels.to(device)\n",
        "        \n",
        "        # 테스트 이미지를 모델에 통과\n",
        "        outputs = net(images)\n",
        "        \n",
        "        # 가장 높은 에너지를 가진 클래스가 예측값이 됩니다.\n",
        "        _, predicted = torch.max(outputs.data, 1)\n",
        "        total += labels.size(0)\n",
        "        correct += (predicted == labels).sum().item()\n",
        "\n",
        "print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.10"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}

NameError: name 'null' is not defined